In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"bhushanasati","key":"64a10884578bccff8e604b2ca2e4ab59"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge

Dataset URL: https://www.kaggle.com/datasets/allen-institute-for-ai/CORD-19-research-challenge
License(s): other
CORD-19-research-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip CORD-19-research-challenge.zip -d cord19

Archive:  CORD-19-research-challenge.zip
replace cord19/COVID.DATA.LIC.AGMT.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
import pandas as pd

# Load the metadata file
metadata_df = pd.read_csv('metadata.csv', low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: 'metadata.csv'

In [ ]:
metadata_df

In [ ]:
metadata_df.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id'],
      dtype='object')

In [ ]:
metadata_df['publish_time'].value_counts()

,count
publish_time,
2021,233709
2020,210884
2022,43121
2015,4147
2016,4105
...,...
1989-09-12,1
1986-08-11,1
2007-08-09,1


In [ ]:
metadata_df.isnull().sum()

,0
cord_uid,0
sha,682894
source_x,0
title,503
doi,399880
pmcid,667089
pubmed_id,557728
license,0
abstract,235544
publish_time,1814


In [ ]:
# Keep rows where both title and abstract are not null
metadata_df = metadata_df[metadata_df['abstract'].notnull() & metadata_df['title'].notnull()]

In [ ]:
metadata_df.reset_index(drop=True, inplace=True)


In [ ]:
metadata_df.drop_duplicates(subset=['title', 'abstract'], inplace=True)


<ipython-input-11-e542867e2d36>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_df.drop_duplicates(subset=['title', 'abstract'], inplace=True)


In [ ]:
metadata_df.drop_duplicates(subset=['title', 'abstract'], inplace=True)

<ipython-input-12-e542867e2d36>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_df.drop_duplicates(subset=['title', 'abstract'], inplace=True)


In [ ]:
metadata_df = metadata_df[['title', 'abstract', 'authors', 'publish_time', 'journal']]

In [ ]:
metadata_df['text'] = metadata_df['title'] + ' ' + metadata_df['abstract']


In [ ]:
metadata_df['text'] = metadata_df['text'].str.lower()


In [ ]:
import re

metadata_df['text'] = metadata_df['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

In [ ]:
metadata_df['text'] = metadata_df['text'].apply(lambda x: re.sub(r'\d+', '', x))

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
metadata_df['tokens'] = metadata_df['text'].apply(nltk.word_tokenize)

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
metadata_df['tokens'] = metadata_df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()
metadata_df['tokens'] = metadata_df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [ ]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
metadata_df['tokens'] = metadata_df['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])

In [ ]:
metadata_df['processed_text'] = metadata_df['tokens'].apply(lambda x: ' '.join(x))


In [ ]:
import numpy as np

num_clients = 10
client_data = []

# Shuffle the data
metadata_df = metadata_df.sample(frac=1).reset_index(drop=True)

# Split data into chunks
data_splits = np.array_split(metadata_df, num_clients)

for i in range(num_clients):
    client_data.append(data_splits[i])


In [ ]:
# Group by a certain topic or keyword
groups = metadata_df.groupby(metadata_df['journal'])

client_data = [group for _, group in groups]


In [ ]:
from transformers import AutoTokenizer
import tensorflow as tf

In [ ]:
def assign_label(text):
    text = text.lower()
    if 'treatment' in text:
        return 'Treatment'
    elif 'vaccine' in text:
        return 'Vaccines'
    # Add additional conditions as needed
    else:
        return 'Others'

metadata_df['label'] = metadata_df['processed_text'].apply(assign_label)


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
metadata_df['label_encoded'] = label_encoder.fit_transform(metadata_df['label'])


In [ ]:
print(metadata_df.columns)


In [ ]:
# Assuming you've created 'label_encoded' in metadata_df
client_data = []

# Shuffle and split the data
shuffled_df = metadata_df.sample(frac=1).reset_index(drop=True)
client_splits = np.array_split(shuffled_df, num_clients)

for split in client_splits:
    client_data.append(split.reset_index(drop=True))


In [ ]:
from transformers import AutoTokenizer
import tensorflow as tf

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

client_datasets = []

for data in client_data:
    # Use the processed text for model input
    texts = list(data['processed_text'])
    labels = list(data['label_encoded'])  # Use the encoded labels

    # Tokenization and Encoding (for models like BERT)
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)

    # Create a dataset object
    client_dataset = tf.data.Dataset.from_tensor_slices((dict(encodings), labels))
    client_datasets.append(client_dataset)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(metadata_df['processed_text'])


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(metadata_df['processed_text'])


In [ ]:
pip install gensim

In [ ]:
from gensim.models import KeyedVectors

# Load pre-trained word vectors (e.g., Word2Vec or GloVe)
word_vectors = KeyedVectors.load_word2vec_format('word2vec.bin', binary=True)

In [ ]:
# Save the preprocessed DataFrame
metadata_df.to_csv('preprocessed_metadata.csv', index=False)


In [ ]:
for idx, data in enumerate(client_data):
    data.to_csv(f'client_{idx}_data.csv', index=False)
